In [ ]:
import numpy as np
import pandas as pd
from datetime import *
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pathlib import Path
import chaospy
import math


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


In [ ]:
model_dir = 'Model_architecture\\FCNN_v1\\'

# Inputs for aerodynamic forces estimation
xref = 0.12192      # Reference point for moment calculation (e.g., aerodynamic center)
chord = 0.4064      # Chord length of the airfoil/wing
x_30 = 0.12192      # x-location at 30% chord
x_50 = 0.2032       # x-location at 50% chord
span = chord*2      # Span of the wing (assuming aspect ratio or geometry)
area = span*chord   # Reference area for aerodynamic coefficients


# Create directories for the results
directory_results = 'Results\\Polars_UQ\\'
Path(directory_results).mkdir(parents=True, exist_ok=True)


# Load dataset
dataset = np.load('Dataset_NN\\dataset.npy')
grid_data = 'Dataset\\grid_data.dat'
CFD_csv_file = 'Dataset\\surface.csv' # File with normals and cell area

# print( '\n-> Shape of the loaded matrix: \n')
# print(dataset.shape) # x, y , z , CP , CF_x, CF_y, CF_z, M , AoA

X = dataset[:,:,[0,1,2,7,8]] # Input
Y = dataset[:,:,3:7]         # Output


## Feature normalisation:
scaler = MinMaxScaler(feature_range=(-1, 1))
samples, points, variables = X.shape
X = np.reshape(X, newshape=(-1, variables))
X = scaler.fit_transform(X)
X = np.reshape(X, newshape=(samples, points, variables))

scalery = MinMaxScaler(feature_range=(-1, 1))
samples, points, variables = Y.shape
Y = np.reshape(Y, newshape=(-1, variables))
Y = scalery.fit_transform(Y)
Y = np.reshape(Y, newshape=(samples, points, variables))

In [ ]:
model = tf.keras.models.load_model(model_dir+ 'neural_network_model.h5')  # Load the model architecture and weights
model.load_weights(model_dir+ 'neural_network_weights.h5')  # Load the model weights separately (if needed)
print(model.summary())  # Print the model summary

# Denormalize
samples, points, variables = X.shape  # Get the shape of the normalized input data
X_test = np.reshape(X, newshape=(-1, variables))  # Flatten the input data for inverse transformation
X_test = scaler.inverse_transform(X_test)  # Inverse transform to original scale
X_test = np.reshape(X_test, newshape=(samples, points, variables))  # Reshape back to original dimensions

cooordinates = pd.read_csv(CFD_csv_file)  # Load coordinates surface data 

In [ ]:
def neural_network(Mach, AoA, npts, X_temp):
  """
  Predicts aerodynamic coefficients (lift and moment) using a trained neural network model.

  Parameters:
  Mach (float): Mach number for prediction.
  AoA (float): Angle of attack in degrees.
  npts (int): Number of points in the input grid.
  X_temp (np.ndarray): Array of shape (npts, 3) containing x, y, z coordinates.

  Returns:
  tuple: Predicted lift and moment coefficients.
  """
  # Create arrays for Mach and AoA, repeated for each point
  Mach_i = np.full((npts, 1), Mach)  # Shape (npts, 1)
  AoA_i = np.full((npts, 1), AoA)    # Shape (npts, 1)

  # Concatenate coordinates with Mach and AoA to form input features
  X_exp = np.concatenate((X_temp, Mach_i, AoA_i), axis=1)  # Shape (npts, 5)

  # Normalize input features using the fitted scaler
  X_exp = scaler.transform(X_exp)
  X_exp = np.expand_dims(X_exp, axis=0)  # Add batch dimension

  # Predict using the neural network model
  Y_exp_pred = model.predict(X_exp, batch_size=1, verbose=0)

  # Inverse transform input features to original scale
  samples, points, variables = X_exp.shape
  X_exp = np.reshape(X_exp, newshape=(-1, variables))
  X_exp = scaler.inverse_transform(X_exp)
  X_exp = np.reshape(X_exp, newshape=(samples, points, variables))

  # Inverse transform predictions to original scale
  samples, points, variables = Y_exp_pred.shape
  Y_exp_pred = np.reshape(Y_exp_pred, newshape=(-1, variables))
  Y_exp_pred = scalery.inverse_transform(Y_exp_pred)
  Y_exp_pred = np.reshape(Y_exp_pred, newshape=(samples, points, variables))

  # Read grid data to get cell areas for force integration
  df = pd.read_csv(grid_data, sep=' ')
  cell_area = np.array(df.Cell_Volume)
  mean_area = np.mean(cell_area)
  cell_area = cell_area / mean_area  # Normalize cell areas

  # Prepare DataFrame with predictions and input features
  df_pred = np.concatenate((X_exp[0, :, :], Y_exp_pred[0, :, :]), axis=1)
  df_pred = pd.DataFrame(df_pred, columns=[
    'x', 'y', 'z', 'Mach', 'AoA',
    'Pressure_Coefficient', 'Skin_Friction_Coefficient_x',
    'Skin_Friction_Coefficient_y', 'Skin_Friction_Coefficient_z'
  ])

  # Extract predicted aerodynamic coefficients
  pressure_dataset = df_pred.Pressure_Coefficient
  cf_x_dataset = df_pred.Skin_Friction_Coefficient_x
  cf_y_dataset = df_pred.Skin_Friction_Coefficient_y
  cf_z_dataset = df_pred.Skin_Friction_Coefficient_z

  # Build dataset dictionary for further processing
  dataset = {
    'PointID': cooordinates.PointID,
    'x': cooordinates.x,
    'y': cooordinates.y,
    'z': cooordinates.z,
    'Pressure_Coefficient': pressure_dataset,
    'Skin_Friction_Coefficient_x': cf_x_dataset,
    'Skin_Friction_Coefficient_y': cf_y_dataset,
    'Skin_Friction_Coefficient_z': cf_z_dataset
  }

  # Convert AoA to radians for trigonometric calculations
  aoa = AoA * np.pi / 180

  # Read grid data again for normal vectors and cell areas
  df_cfd = pd.read_csv(grid_data, sep=' ')
  nn_data = pd.DataFrame(data=dataset)

  def compute_aero_coeff(ds):
    """
    Computes aerodynamic coefficients (lift, drag, moment) from predicted data.

    Parameters:
    ds (pd.DataFrame): DataFrame containing coordinates and predicted coefficients.

    Returns:
    tuple: lift, drag, and moment coefficients.
    """
    coordinates = [ds.x, ds.y, ds.z]  # Coordinates arrays
    shear = [ds.Skin_Friction_Coefficient_x, ds.Skin_Friction_Coefficient_y, ds.Skin_Friction_Coefficient_z]  # Shear coefficients
    cp = ds.Pressure_Coefficient  # Pressure coefficient
    normal = [df_cfd.X_Grid_K_Unit_Normal, df_cfd.Y_Grid_K_Unit_Normal, df_cfd.Z_Grid_K_Unit_Normal]  # Surface normals
    cell_area = df_cfd.Cell_Volume  # Cell areas

    # Calculate force components per area
    taux = (shear[0] - normal[0] * cp) / area
    tauz = (shear[2] - normal[2] * cp) / area

    # Integrate force components over the surface
    fz = np.sum(tauz * cell_area)
    fx = np.sum(taux * cell_area)

    # Calculate lift and drag using AoA
    lift = fz * np.cos(aoa) - fx * np.sin(aoa)
    drag = fx * np.cos(aoa) + fz * np.sin(aoa)

    # Calculate moment coefficient
    my = (coordinates[2] * (taux * np.cos(aoa) + tauz * np.sin(aoa)) -
        (coordinates[0] - xref) * (tauz * np.cos(aoa) - taux * np.sin(aoa))) / chord
    moment = np.sum(my * cell_area)
    return lift, drag, moment

  # Compute lift and moment from predictions
  lift_NN, drag_NN, moment_NN = compute_aero_coeff(nn_data)

  return lift_NN, moment_NN  # Return lift and moment coefficients

In [ ]:
########################################
### PREDICTION FOR POLARS UQ
########################################


Mach_polars = [0.74 , 0.76 , 0.78 , 0.80 , 0.82 , 0.84]  # List of Mach numbers for the polars
AoA_polars = np.arange(0.0,5.1,0.1)  # Range of angle of attack values from 0 to 5 degrees

for Mach in Mach_polars:  # Loop over each Mach number
  lift_list = []  # Initialize list to store lift coefficients
  for AoA in AoA_polars:  # Loop over each angle of attack
    lift_temp = neural_network(Mach, AoA)  # Predict lift and moment using the neural network
    lift_list.append(lift_temp)  # Append the predicted lift (and moment) to the list

  fig, ax = plt.subplots(1, 1, figsize=(18.5, 10.5))
  title_plot = 'Lift Polar - M = '+str(Mach)
  print('\n' + title_plot)
  fig.suptitle(title_plot, fontsize=16)
  ax.plot(AoA_polars,np.array(lift_list),'k-',linewidth=2)  
  ax.set_xlabel(r'$AoA \,\, [Deg]$', fontsize=16)
  ax.set_ylabel(r'$Lift \,\, Coefficient$', fontsize=16)
  name_fig = 'Lift_polar_Mach_'+str(Mach)
  plt.show()


In [ ]:
########################################
### PREDICTION FOR POLARS UQ
########################################
print("\n")
print('PREDICTION FOR POLARS UQ')
print("\n")

Mach_list = [0.84, 0.84, 0.84, 0.8, 0.8, 0.8, 0.76, 0.76, 0.76]  # List of Mach numbers for each subplot
Mach_range = 0.01  # Standard deviation for Mach number distribution

AoA_list = [1.0, 3.0, 5.0, 1.0, 3.0, 5.0, 1.0, 3.0, 5.0]  # List of AoA values for each subplot
AoA_range = 0.1  # Standard deviation for AoA distribution

random_variables = 2  # Number of random variables (Mach and AoA)
poly_order = 5  # Polynomial order for expansion (used for num_samples calculation)
num_samples = int(math.factorial(random_variables + poly_order) / (math.factorial(random_variables) * math.factorial(poly_order)))  # Number of samples for regression

poly_order = 4  # Polynomial order for the actual expansion

plt.rcParams.update({'font.size': 24})  # Set global font size for plots
fig = plt.figure(figsize=(18.5, 10.5))  # Create a new figure
gs = fig.add_gridspec(5, 5, width_ratios=[0.05, 1, 1, 1, 0.05], height_ratios=[0.05, 1, 1, 1, 0.1], wspace=0.5, hspace=1.0)  # Define grid layout

ax_main = fig.add_subplot(gs[:, :])  # Main axis for labels
ax_main.set_ylabel(r'$Mach$',labelpad=20)  # Set y-axis label
ax_main.set_xlabel(r'$Angle \, of \, attack \,\, [deg]$',labelpad=20)  # Set x-axis label
ax_main.set_ylim([0,10])  # Set y-axis limits
ax_main.set_yticks([2,5,8])  # Set y-tick positions
ax_main.set_yticklabels(['0.76', '0.80', '0.84'])  # Set y-tick labels
ax_main.set_xlim([0,10])  # Set x-axis limits
ax_main.set_xticks([2,5,8])  # Set x-tick positions
ax_main.set_xticklabels([ '1.0', '3.0', '5.0'])  # Set x-tick labels

subplots = []  # List to store individual subplot axes

for i, (Mach, AoA) in enumerate(zip(Mach_list, AoA_list)):
    row = i // 3  # Calculate row index in grid
    col = i % 3   # Calculate column index in grid

    title_plot = 'M = {} - AoA = {}'.format(Mach, AoA)  # Title for current subplot
    print('\n' + title_plot)

    Mach_dist = chaospy.Normal(Mach, Mach_range)  # Define normal distribution for Mach
    AoA_dist = chaospy.Normal(AoA, AoA_range)     # Define normal distribution for AoA
    joint = chaospy.J(Mach_dist, AoA_dist)        # Create joint distribution

    expansion = chaospy.generate_expansion(poly_order, joint)  # Generate polynomial chaos expansion

    samples = joint.sample(num_samples, rule="latin_hypercube", seed=1)  # Sample input space

    evaluations = np.array([neural_network(Mach_sample, AoA_sample) for Mach_sample, AoA_sample in samples.T])  # Evaluate NN for each sample

    approx_solver = chaospy.fit_regression(expansion, samples, evaluations)  # Fit regression model

    num_samples_eval = 10000  # Number of samples for evaluation

    samples_eval = joint.sample(num_samples_eval, rule="latin_hypercube", seed=1)  # Generate evaluation samples

    approximations = chaospy.call(approx_solver, samples_eval)  # Get approximated outputs
    binss = 100  # Number of bins for histogram
    bin_edges = np.linspace(approximations.min(), approximations.max(), binss)  # Bin edges for histogram

    ax = fig.add_subplot(gs[row + 1, col + 1])  # Create subplot at correct grid position
    ax.hist(approximations, bins=bin_edges, density=True, edgecolor='black', color='grey',
            alpha=0.7)  # Plot histogram of approximations
    # ax.legend(loc='upper left', fontsize=10)
    ax.set_xlabel(r'$C_L$')  # Set x-axis label
    ax.set_ylabel(r'$PDF$')  # Set y-axis label

    subplots.append(ax)  # Add the subplot axes to the list

# Call the desired subplot outside the for loop
subplots[0].set_xlim([-0.1, 0.4])      # Set x-axis limits for subplot 0
subplots[1].set_xlim([-0.07, 0.52])    # Set x-axis limits for subplot 1
subplots[3].set_xlim([0.24, 0.355])    # Set x-axis limits for subplot 3
subplots[4].set_xlim([0.38, 0.56])     # Set x-axis limits for subplot 4
subplots[5].set_xlim([0.3, 0.9])       # Set x-axis limits for subplot 5

plt.show()  # Display the figure


In [ ]:
########################################
### PREDICTION FOR POLARS UQ
########################################
print("\n")
print('PREDICTION FOR POLARS UQ')
print("\n")

Mach_list = [0.74, 0.76, 0.80, 0.82, 0.84 ]  # List of Mach numbers to analyze
Mach_range = 0.01  # Standard deviation for Mach number uncertainty

AoA_list = [1.0, 3.0, 5.0 ]  # List of angles of attack to analyze
AoA_range = 0.1  # Standard deviation for AoA uncertainty

random_variables = 2  # Number of uncertain input variables (Mach, AoA)
poly_order = 5  # Maximum polynomial order for expansion (for num_samples calculation)
num_samples = int(math.factorial(random_variables + poly_order) / (math.factorial(random_variables) * math.factorial(poly_order)))  # Number of samples for regression

poly_ord_min = 1  # Minimum polynomial order for subplots
poly_ord_max = 5  # Maximum polynomial order for subplots

for Mach in Mach_list:  # Loop over Mach numbers
    for AoA in AoA_list:  # Loop over AoA values
        # Create a figure with subplots for each polynomial order
        plt.rcParams.update({'font.size': 24})  # Set font size for plots
        fig, axs = plt.subplots((poly_ord_max+1 - poly_ord_min), 1, figsize=(18.5, 10.5))  # Create subplots for each poly order
        title_plot = 'UQ Polars - M = '+str(Mach)+' - AoA = '+str(AoA)
        print('\n' + title_plot)
        fig.suptitle(title_plot, fontsize=16)  # Set figure title

        Mach_dist = chaospy.Normal(Mach, Mach_range)  # Define normal distribution for Mach
        AoA_dist = chaospy.Normal(AoA, AoA_range)  # Define normal distribution for AoA
        joint = chaospy.J(Mach_dist, AoA_dist)  # Create joint distribution

        expansion = chaospy.generate_expansion(poly_ord_max, joint)  # Generate polynomial chaos expansion

        samples = joint.sample(num_samples, rule="latin_hypercube", seed=1)  # Sample input space

        evaluations = np.array([neural_network(Mach_sample, AoA_sample) for Mach_sample, AoA_sample in samples.T])  # Evaluate NN for each sample

        approx_solver = chaospy.fit_regression(expansion, samples, evaluations)  # Fit regression model

        num_samples_eval = 10000  # Number of samples for evaluation

        samples_eval = joint.sample(num_samples_eval, rule="latin_hypercube", seed=1)  # Generate evaluation samples

        approximations = chaospy.call(approx_solver, samples_eval)  # Get approximated outputs
        binss = 500  # Number of bins for histogram
        bin_edges = np.linspace(approximations.min(), approximations.max(), binss)  # Bin edges for histogram

        # Iterate over each poly_order value
        for i, poly_order in enumerate(range(poly_ord_min, poly_ord_max+1)):
                Mach_dist = chaospy.Normal(Mach, Mach_range)  # Redefine Mach distribution for current poly order
                AoA_dist = chaospy.Normal(AoA, AoA_range)  # Redefine AoA distribution for current poly order
                joint = chaospy.J(Mach_dist, AoA_dist)  # Create joint distribution

                expansion = chaospy.generate_expansion(poly_order, joint)  # Generate polynomial chaos expansion

                samples = joint.sample(num_samples, rule="latin_hypercube", seed=1)  # Sample input space

                evaluations = np.array([neural_network(Mach_sample, AoA_sample) for Mach_sample, AoA_sample in samples.T])  # Evaluate NN for each sample

                approx_solver = chaospy.fit_regression(expansion, samples, evaluations)  # Fit regression model

                num_samples_eval = 10000  # Number of samples for evaluation

                samples_eval = joint.sample(num_samples_eval, rule="latin_hypercube", seed=1)  # Generate evaluation samples

                approximations = chaospy.call(approx_solver, samples_eval)  # Get approximated outputs

                axs[i].hist(approximations, bins=bin_edges, density=True, edgecolor='black', color='grey', alpha=0.7, label=f'Poly Order: {poly_order}')  # Plot histogram
                axs[i].legend(loc='upper left')  # Show legend
                axs[i].set_ylabel(r'$PDF$')  # Set y-axis label

        axs[i].set_xlabel(r'$C_L$')  # Set x-axis label for the last subplot
        
        approx_min = 0.4 #np.min(approximations)  # Set minimum x-axis value (can be adjusted)
        approx_max = 0.6 #np.max(approximations)  # Set maximum x-axis value (can be adjusted)

        # Set a global min and max value for the x-axis across all subplots
        for j, ax in enumerate(axs):
                if j < len(axs) - 1:
                        ax.set_xticks([])  # Hide x-ticks for all but last subplot
                ax.set_xlim(approx_min, approx_max)  # Set x-axis limits

        plt.tight_layout()  # Adjust subplot layout
        plt.show()  # Display the figure


In [ ]:
########################################
### SOBOL INDICES
########################################

print("\n")
print('SOBOL INDICES')
print("\n")

poly_order = 4  # Set polynomial order for chaos expansion

for Mach in Mach_list:  # Loop over Mach numbers
    fig1, axs1 = plt.subplots(1, 1, figsize=(9.25, 5.25))  # Create a new figure for each Mach
    title_plot_sobol = 'Sobol indices - M = '+str(Mach)
    fig1.suptitle(title_plot_sobol, fontsize=16)  # Set figure title

    sobol_indices_mach = []  # List to store Sobol indices for Mach
    sobol_indices_aoa = []   # List to store Sobol indices for AoA

    for AoA in AoA_list:  # Loop over AoA values
        Mach_dist = chaospy.Normal(Mach, Mach_range)  # Define normal distribution for Mach
        AoA_dist = chaospy.Normal(AoA, AoA_range)     # Define normal distribution for AoA
        joint = chaospy.J(Mach_dist, AoA_dist)        # Create joint distribution

        expansion = chaospy.generate_expansion(poly_order, joint)  # Generate polynomial chaos expansion

        samples = joint.sample(num_samples, rule="latin_hypercube", seed=1)  # Sample input space

        evaluations = np.array([neural_network(Mach_sample, AoA_sample) for Mach_sample, AoA_sample in samples.T])  # Evaluate NN for each sample

        approx_solver = chaospy.fit_regression(expansion, samples, evaluations)  # Fit regression model

        num_samples_eval = 10000  # Number of samples for evaluation

        samples_eval = joint.sample(num_samples_eval, rule="latin_hypercube", seed=1)  # Generate evaluation samples

        approximations = chaospy.call(approx_solver, samples_eval)  # Get approximated outputs

        # Calculate Sobol indices
        sobol_indices = chaospy.Sens_m(approx_solver, joint, samples=samples_eval)  # Compute Sobol indices

        sobol_indices_mach.append(sobol_indices[0])  # Store Mach Sobol index
        sobol_indices_aoa.append(sobol_indices[1])   # Store AoA Sobol index

    labels = [str(AoA) for AoA in AoA_list]  # Create labels for AoA values

    axs1.bar(labels, sobol_indices_mach, width=0.6, alpha=0.5, label=r'$Mach$')  # Plot Mach Sobol indices
    axs1.bar(labels, sobol_indices_aoa, width=0.6, alpha=0.5, label=r'$Angle \, of \, attack$', bottom=sobol_indices_mach)  # Plot AoA Sobol indices stacked
    axs1.set_xlabel(r'$AoA \,\, [Deg]$', fontsize=16)  # Set x-axis label
    # axs1.set_ylabel(r'$Sobol \,\, Index$', fontsize=16)
    axs1.legend(loc='upper right', fontsize=16)  # Show legend
    axs1.set_ylim(0.0, 1.1)  # Set y-axis limits
    plt.tight_layout()  # Adjust layout
    plt.show()  # Display the plot


In [ ]:
print("\n")
print('SOBOL INDICES')
print("\n")
Mach_list = [0.84, 0.8, 0.76]  # List of Mach numbers to analyze
AoA_list = [1, 3, 5]           # List of angles of attack to analyze
poly_order = 3                 # Polynomial order for chaos expansion
plt.rcParams.update({'font.size': 24})  # Set global font size for plots
fig = plt.figure(figsize=(18.5, 10.5))  # Create a new figure
gs = fig.add_gridspec(5, 5, width_ratios=[0.05, 0.8, 0.8, 0.8, 0.05], height_ratios=[0.05, 2.0, 2.0, 2.0, 0.1], wspace=0.5, hspace=0.8)  # Define grid layout

ax_main = fig.add_subplot(gs[:, :])  # Main axis for labels
ax_main.set_ylabel(r'$Mach$', labelpad=20)  # Set y-axis label
ax_main.set_xlabel(r'$Angle \, of \, attack \,\, [deg]$', labelpad=20)  # Set x-axis label
ax_main.set_ylim([0, 10])  # Set y-axis limits
ax_main.set_yticks([2, 5, 8])  # Set y-tick positions
ax_main.set_yticklabels(['0.76', '0.80', '0.84'])  # Set y-tick labels
ax_main.set_xlim([0, 10])  # Set x-axis limits
ax_main.set_xticks([2, 5, 8])  # Set x-tick positions
ax_main.set_xticklabels(['1.0', '3.0', '5.0'])  # Set x-tick labels

legend_labels = []  # List to store legend handles

for i, Mach in enumerate(Mach_list):  # Loop over Mach numbers
    for j, AoA in enumerate(AoA_list):  # Loop over AoA values
        sub_ax = fig.add_subplot(gs[i+1, j+1])  # Create subplot for each Mach/AoA combination

        sobol_indices_mach = []  # List to store Mach Sobol index
        sobol_indices_aoa = []   # List to store AoA Sobol index

        Mach_dist = chaospy.Normal(Mach, Mach_range)  # Define normal distribution for Mach
        AoA_dist = chaospy.Normal(AoA, AoA_range)     # Define normal distribution for AoA
        joint = chaospy.J(Mach_dist, AoA_dist)        # Create joint distribution

        expansion = chaospy.generate_expansion(poly_order, joint)  # Generate polynomial chaos expansion

        samples = joint.sample(num_samples, rule="latin_hypercube", seed=1)  # Sample input space

        evaluations = np.array([neural_network(Mach_sample, AoA_sample) for Mach_sample, AoA_sample in samples.T])  # Evaluate NN for each sample

        approx_solver = chaospy.fit_regression(expansion, samples, evaluations)  # Fit regression model

        num_samples_eval = 10000  # Number of samples for evaluation

        samples_eval = joint.sample(num_samples_eval, rule="latin_hypercube", seed=1)  # Generate evaluation samples

        approximations = chaospy.call(approx_solver, samples_eval)  # Get approximated outputs

        # Calculate Sobol indices
        sobol_indices = chaospy.Sens_m(approx_solver, joint, samples=samples_eval)  # Compute Sobol indices
        sobol_indices_mach.append(sobol_indices[0])  # Store Mach Sobol index
        sobol_indices_aoa.append(sobol_indices[1])   # Store AoA Sobol index

        mach_bar = sub_ax.bar([AoA], sobol_indices_mach, color='grey', width=0.6, alpha=0.5, label=r'$Mach$')  # Plot Mach Sobol index
        aoa_bar = sub_ax.bar([AoA], sobol_indices_aoa, color='black', width=0.6, alpha=0.5, label=r'$Angle \, of \, attack$', bottom=sobol_indices_mach)  # Plot AoA Sobol index stacked
        # sub_ax.set_xlabel(r'$AoA \,\, [Deg]$', fontsize=12)
        sub_ax.set_ylabel(r'$Sobol \,\, Idx$', fontsize=22)  # Set y-axis label
        sub_ax.set_ylim(0.0, 1.1)  # Set y-axis limits
        sub_ax.set_xticks([])  # Hide x-ticks

        # Add label to legend only once
        if (i == 0) and (j == 2):  # Only add legend handles once
            legend_labels.append(mach_bar[0])
            legend_labels.append(aoa_bar[0])

ax_main.legend(legend_labels, ['Mach', 'Angle of attack'], loc='upper right', ncol=2 ,fontsize=22)  # Add legend to main axis
plt.show()  # Display the figure
